# Introduction
<font color = green> <strong>MUST READ:</strong></font> This Notebook tutorial will allow you to register a TNG manifest file into Google Cloud Platform (GCP) account to enable device connectivity. To run this Notebook successfuly, its required to have GCP credentials updated in this Notebook, Manifest file and Manifest file Signer are generated, and Manifest file is Signed by Manifest signer. The Manifest file related steps will be taken care by TNGTLS Manifest File Generation Notebook.

Before running this Notebook
1. Have GCP account credentials located in docs folder as GCP_test_account_credentials.csv
2. Have manifest and dataview credentials located in Docs folder as iot-manifest.json and data-view.json
3. Run TNGTLS Manifest File Generation Notebook to ensure Manifest file generated and signed

### Prerequisites:
This step of the tutorial will attempt to load all the necessary modules and their dependencies on your machine. If the modules are already installed you can safely step over the next Tutorial step.

<font color = orange> <strong>Note</strong></font> 
1. Installation time for prerequisites depends upon system and network speed.
2. Installing prerequisites for the first time takes more time and watch the kernel status for progress. Following image helps to locate the Kernel status,
<center><img src="../../../assets/notebook/img/kerner_status.png" alt="**Check Kernel Status**" /></center>


3. Installing prerequisites gives the following error and it can be safely ignored. Functionality remains unaffected.
    - <font color = orange> azure-cli 2.0.76 has requirement colorama~=0.4.1, but you'll have colorama 0.3.9 which is incompatible.</font>
    - <font color = orange> azure-cli 2.0.76 has requirement pytz==2019.1, but you'll have pytz 2019.3 which is incompatible. </font>

In [ ]:
import sys, os

home_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.realpath(os.getcwd()))))
module_path = os.path.join(home_path, 'assets', 'python')
if not module_path in sys.path:
    sys.path.append(module_path)

from requirements_helper import requirements_installer
obj = requirements_installer(os.path.join(home_path, 'assets', 'requirements.txt'))

### Load the necessary modules and helper functions
This step loads the required modules and helper functions to perform the required operations.

In [ ]:
import json
from at_gui import *
from load_manifest import *
from IPython.display import display
from ipywidgets import *

The following steps assist to perform
1. Manifest file upload to GCP account
2. Triggering the GCP-GUI to control connected device

#### File navigation for Manifest file and certificate (1. Upload Manifest file)
This button helps to load the Manifest file to GCP account. To be able to upload successfully, its required to provide devices Manifest file and Manifest file Signer certificate. 

#### Trigger GCP GUI (2. GCP GUI)
Before running this step, target hardware should be compiled with the above host name and wifi credentials. Once, the target hardware is programmed and connected to cloud, click this button to pops up a GUI. This GUI can be used to monitor the LED status.

In [ ]:
manifest_credential_file = os.path.join(home_path, 'docs', 'iot-manifest.json') #'../../../docs/iot-manifest.json'
dataview_credential_file = os.path.join(home_path, 'docs', 'data-view.json') #'../../../docs/data-view.json'

manifest_file = FileUpload(description='Step1a. Load Manifest JSON File', accept='.json', layout=widgets.Layout(width='auto'), multiple=False)
validation_cert = FileUpload(description='Step1b. Load Validation CERT File', accept='.crt', layout=widgets.Layout(width='auto'), multiple=False)
manifest_upload = widgets.Button(description = "Step1c. Upload Manifest File", layout=widgets.Layout(width='auto'))
v = widgets.VBox((manifest_file, validation_cert, manifest_upload))

def upload_manifest_callback(b):
    if not manifest_file.data or not validation_cert.data:
        print('Step1a & Step1b should be executed first before proceeding to this step\r\n')
        return None
    manifest_data = json.loads(manifest_file.data[0])
    validation_certificate = validation_cert.data[0]
    if not os.path.isfile(manifest_credential_file):
        print("iot-manifest.json file not present in docs folder.The file should be created with proper credentials\r\n")
        return None
    if gcp_upload_manifest(manifest_credential_file, validation_certificate, manifest_data) :
        manifest_upload.button_style = 'success'
    else:
        manifest_upload.button_style = 'danger'
def execute_gcp_gui(b):
    if not os.path.isfile(dataview_credential_file) :
        print("data-view.json file not present in docs folder.The file should be created with GCP account\r\n")
        return None
    run_gcp_gui(dataview_credential_file)
    gcp_gui.button_style = 'success'
    
manifest_upload.on_click(upload_manifest_callback)
gcp_gui = widgets.Button(description = "GCP GUI")
gcp_gui.on_click(execute_gcp_gui)

display(widgets.VBox((manifest_file, validation_cert, manifest_upload)))
print('--------------------------------------')
print('Before clicking GCP GUI its required to have Manifest file uploaded and \
Connecting embedded project to cloud by using host details and wifi credentials. \
Click below GCP GUI button ONLY after establishing connection with Cloud')
display(gcp_gui) 
print('--------------------------------------')